# Setup

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
import requests
from io import BytesIO
from openai import OpenAI

client = OpenAI()

# RAG client

In [12]:
class RAG_Client():
    def __init__(self):
        self.vector_store = client.vector_stores.create()
        self.client = OpenAI()
        
    def load_file(self, file_path: str):
        if file_path.startswith("http://") or file_path.startswith("https://"):
            # Download the file content from the URL
            response = requests.get(file_path)
            file_content = BytesIO(response.content)
            file_name = file_path.split("/")[-1]
            file_tuple = (file_name, file_content)
            result = client.files.create(
                file=file_tuple,
                purpose="assistants"
            )
        else:
            # Handle local file path
            with open(file_path, "rb") as file_content:
                result = client.files.create(
                    file=file_content,
                    purpose="assistants"
                )
                
        file_id = result.id
        client.vector_stores.files.create(
            vector_store_id=self.vector_store.id,
            file_id=file_id
        )

    def check_loading_completion(self):
        result = self.client.vector_stores.files.list(
            vector_store_id=self.vector_store.id
        )
        if result.data[0].status == "completed":
            return True
            
        return False

    def generate_with_file_search(self, query: str):
        if not self.check_loading_completion():
            return "File loading is not complete yet, please wait."
            
        response = self.client.responses.create(
            model="gpt-4o-mini",
            input=query,
            tools=[{
                "type": "file_search",
                "vector_store_ids": [self.vector_store.id],
            }]
        )

        return response

    

# Vector store operations

In [102]:
def vector_store_create(name: None | str=None):
    vector_store = client.vector_stores.create(name=name)

    return vector_store.id

def vector_store_delete(vector_store_id: str):
    try:
        client.vector_stores.delete(
            vector_store_id=vector_store_id
        )
        return True
    except Exception as e:
        print("Error occured: ", e)

def vector_store_list():
    return client.vector_stores.list().model_dump()

def vector_store_retrieve(vector_store_id: str):
    return client.vector_stores.retrieve(
        vector_store_id=vector_store_id
    )

def vector_store_check_completion(vector_store_id: str):
    if vector_store_retrieve(vector_store_id).status == "completed":
        return True
    else:
        return False

def vector_store_status(vector_store_id: str):
    return vector_store_retrieve(vector_store_id).status

def vector_store_update(vector_store_id, *args, **kwargs):
    client.vector_stores.update(
        vector_store_id=vector_store_id,
        *args,
        **kwargs
    )

# OpenAI file operations

In [39]:
def file_upload_openai(file_path: str):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
            
    file_id = result.id

    return file_id


def batch_upload_openai(file_paths: list[str]):
    file_ids = []
    for file_path in file_paths:
        if file_path.startswith("http://") or file_path.startswith("https://"):
            # Download the file content from the URL
            response = requests.get(file_path)
            file_content = BytesIO(response.content)
            file_name = file_path.split("/")[-1]
            file_tuple = (file_name, file_content)
            result = client.files.create(
                file=file_tuple,
                purpose="assistants"
            )
        else:
            # Handle local file path
            with open(file_path, "rb") as file_content:
                result = client.files.create(
                    file=file_content,
                    purpose="assistants"
                )
                
        file_ids.append(result.id)

    return file_ids


def file_list_openai():
    return client.files.list().model_dump()


def file_delete_openai(file_id: str):
    try:
        return client.files.delete(file_id)
    except Exception as e:
        print("Error occured:", e)
        return False

def batch_delete_openai(file_ids: list[str]):
    count = 0
    for file_id in file_ids:
        try:
            client.files.delete(file_id)
            count += 1
        except Exception as e:
            print("Error occured:", e)

    return f"{count} out of {len(file_ids)} files deleted successfully."

# Vector store file operations

In [97]:
def vector_store_file_create(file_id: str, vector_store_id: str):
    return client.vector_stores.files.create(
        file_id=file_id,
        vector_store_id=vector_store_id,
    )

def vector_store_file_creation_status(file_id: str, vector_store_id: str):
    return file_retrieve(file_id, vector_store_id).status
    
def vector_store_file_creation_check_completion(file_id: str, vector_store_id: str):
    if file_retrieve(file_id, vector_store_id).status == "completed":
        return True
    return False
    

def vector_store_file_create_pole(file_id: str, vector_store_id: str):
    return client.vector_stores.files.create_and_poll(
        file_id=file_id,
        vector_store_id=vector_store_id,
    )


def vector_store_file_upload_pole(file_path: str, vector_store_id: str):
    return client.vector_stores.files.upload_and_poll(
        vector_store_id=vector_store_id,
        file=open(file_path, "rb")
    ).id


def vector_store_file_retrieve(file_id: str, vector_store_id: str):
    try:
        return client.vector_stores.files.retrieve(
            vector_store_id=vector_store_id,
            file_id=file_id
        )
    except Exception as e:
        return ("Error occured: ", e)

def vector_store_file_update(file_id: str, vector_store_id: str, attributes: dict):
    client.vector_stores.files.update(
        vector_store_id=vector_store_id,
        file_id=file_id,
        attributes=attributes
    )


def vector_store_file_delete(file_id: str, vector_store_id: str):
    try:
        return client.vector_stores.files.delete(
            vector_store_id=vector_store_id,
            file_id=file_id
        )
    except Exception as e:
        return f"Err: {e}"


def vector_store_file_list(vector_store_id: str):
    return client.vector_stores.files.list(
        vector_store_id=vector_store_id
    ).model_dump()

## batch operations

In [5]:
def batch_create(vector_store_id: str, file_ids: list[str]):
    return client.vector_stores.file_batches.create(
        vector_store_id=vector_store_id,
        file_ids=file_ids
    ).id
    
    
def batch_create_and_pole(vector_store_id: str, file_ids: list[str]):
    batch_id = client.vector_stores.file_batches.create_and_poll(
        vector_store_id=vector_store_id,
        file_ids=file_ids
    ).id

    return batch_id


def batch_retrieve(batch_id: str, vector_store_id: str):
    return client.vector_stores.file_batches.retrieve(
        batch_id=batch_id,
        vector_store_id=vector_store_id,
    )

def batch_creation_check_completion(batch_id: str, vector_store_id: str):
    if batch_retrieve(batch_id, vector_store_id).status == "completed":
        return True
    return False

def batch_creation_get_status(batch_id: str, vector_store_id: str):
    return batch_retrieve(batch_id, vector_store_id).status

def batch_cancelation(batch_id: str, vector_store_id: str):
    return client.vector_stores.file_batches.cancel(
        batch_id=batch_id,
        vector_store_id=vector_store_id,
    )

def batch_list(vector_store_id: str):
    return client.vector_stores.file_batches.list(
        vector_store_id=vector_store_id,
    )

# Genaration

In [145]:
def generate_with_file_search(query: str, vector_store_id: str):
    client = OpenAI()
    response = client.responses.create(
        model="gpt-4o-mini",
        input=query,
        tools=[{
            "type": "file_search",
            "vector_store_ids": [vector_store_id],
        }]
    )

    return response

def format_results(results):
    formatted_results = ''
    for result in results.data:
        formatted_result = f"<result file_id='{result.file_id}' file_name='{result.filename}'>"
        for part in result.content:
            formatted_result += f"<content>{part.text}</content>"
        formatted_results += formatted_result + "</result>"
    return f"<sources>{formatted_results}</sources>"
    
def semantic_search(query: str, vector_store_id: str):
    results = client.vector_stores.search(
        vector_store_id=vector_store_id,
        query=query,
    )
    
    return results

def generate_with_information_retrieval(query: str, vector_store_id: str):
    results = semantic_search(query, vector_store_id)
    formatted_results = format_results(results)
    print("="*30)
    print(formatted_results)
    print("="*30)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "assistant",
                "content": "Produce a concise answer to the query based on the provided sources. If you don't have access to related information in your context just say \" I don't know\""
            },
            {
                "role": "user",
                "content": f"Sources: {formatted_results}\n\nQuery: '{query}'"
            }
        ],
    )

    return response

# Usage

## vector store operations

In [12]:
vector_store_id = vector_store_create()
vector_store_id

'vs_6829a2598a308191a2cdee158d5961e3'

In [13]:
vector_store_retrieve(vector_store_id)

VectorStore(id='vs_6829a2598a308191a2cdee158d5961e3', created_at=1747559001, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1747559001, metadata={}, name=None, object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)

In [16]:
vector_store_update(vector_store_id=vector_store_id, name="my_vector_store")

In [17]:
vector_store_retrieve(vector_store_id).name

'my_vector_store'

In [103]:
vector_store_status(vector_store_id)

'completed'

In [99]:
vector_store_check_completion(vector_store_id)

True

In [20]:
vector_store_delete(vector_store_id)

True

In [21]:
vector_store_delete(vector_store_id)

Error occured:  Error code: 404 - {'error': {'message': "No vector store found with id 'vs_6829a2598a308191a2cdee158d5961e3'.", 'type': 'invalid_request_error', 'param': 'vector_store_id', 'code': None}}


In [19]:
vector_store_list()

{'data': [{'id': 'vs_6829a2598a308191a2cdee158d5961e3',
   'created_at': 1747559001,
   'file_counts': {'cancelled': 0,
    'completed': 0,
    'failed': 0,
    'in_progress': 0,
    'total': 0},
   'last_active_at': 1747559085,
   'metadata': {},
   'name': 'my_vector_store',
   'object': 'vector_store',
   'status': 'completed',
   'usage_bytes': 0,
   'expires_after': None,
   'expires_at': None},
  {'id': 'vs_6829a218d8988191a7a5720dd2d1c098',
   'created_at': 1747558936,
   'file_counts': {'cancelled': 0,
    'completed': 0,
    'failed': 0,
    'in_progress': 0,
    'total': 0},
   'last_active_at': 1747558936,
   'metadata': {},
   'name': None,
   'object': 'vector_store',
   'status': 'completed',
   'usage_bytes': 0,
   'expires_after': None,
   'expires_at': None},
  {'id': 'vs_6829880083a88191aaf7788961e19186',
   'created_at': 1747552256,
   'file_counts': {'cancelled': 0,
    'completed': 0,
    'failed': 0,
    'in_progress': 0,
    'total': 0},
   'last_active_at': 1747

## openai file operations

In [30]:
file_path = "https://cdn.openai.com/API/docs/deep_research_blog.pdf"
file_paths = [
    "https://cdn.openai.com/API/docs/deep_research_blog.pdf",
    "https://www.nitrd.gov/nitrdgroups/images/c/c1/American-AI-Initiative-One-Year-Annual-Report.pdf",
    "https://ww2.arb.ca.gov/sites/default/files/auction-proceeds/cci_annual_report_2025.pdf"
]

In [35]:
file_id = file_upload_openai(file_path)
file_id

'file-MddCbCpSNzSR8SZ1XRqeFf'

In [36]:
file_delete_openai(file_id)

FileDeleted(id='file-MddCbCpSNzSR8SZ1XRqeFf', deleted=True, object='file')

In [37]:
file_delete_openai(file_id)

Error occured: Error code: 404 - {'error': {'message': 'No such File object: file-MddCbCpSNzSR8SZ1XRqeFf', 'type': 'invalid_request_error', 'param': 'id', 'code': None}}


False

In [40]:
list_files = file_list_openai()
list_files

{'data': [{'id': 'file-SWML324Mjaw6o15CNnbaXC',
   'bytes': 8573289,
   'created_at': 1747550007,
   'filename': 'deep_research_blog.pdf',
   'object': 'file',
   'purpose': 'assistants',
   'status': 'processed',
   'expires_at': None,
   'status_details': None},
  {'id': 'file-Rz1MyK1eji78JwH7j9J3sA',
   'bytes': 8573289,
   'created_at': 1747549978,
   'filename': 'deep_research_blog.pdf',
   'object': 'file',
   'purpose': 'assistants',
   'status': 'processed',
   'expires_at': None,
   'status_details': None},
  {'id': 'file-HXHAxP9Y27EaWiNxF8FKpv',
   'bytes': 8573289,
   'created_at': 1747549913,
   'filename': 'deep_research_blog.pdf',
   'object': 'file',
   'purpose': 'assistants',
   'status': 'processed',
   'expires_at': None,
   'status_details': None},
  {'id': 'file-CCMoyvub6FNPiKTSD7s1fq',
   'bytes': 8573289,
   'created_at': 1747547915,
   'filename': 'deep_research_blog.pdf',
   'object': 'file',
   'purpose': 'assistants',
   'status': 'processed',
   'expires_at'

In [41]:
file_ids = batch_upload_openai(file_paths)
file_ids

['file-DprfWNMdHJtBzkKdraDZAx',
 'file-Tgf1CcLa5r7VcVNLbwaNmJ',
 'file-8zETMDPgMtmGWE4FCEAK7z']

In [42]:
file_ids = [
    "file-SWML324Mjaw6o15CNnbaXC",
    "file-Rz1MyK1eji78JwH7j9J3sA"
]
batch_delete_openai(file_ids)

'2 out of 2 files deleted successfully.'

## vector store file operations

In [68]:
file_path = "https://cdn.openai.com/API/docs/deep_research_blog.pdf"
file_paths = [
    "https://cdn.openai.com/API/docs/deep_research_blog.pdf",
    "https://www.nitrd.gov/nitrdgroups/images/c/c1/American-AI-Initiative-One-Year-Annual-Report.pdf",
    "https://ww2.arb.ca.gov/sites/default/files/auction-proceeds/cci_annual_report_2025.pdf"
]

# file_ids = batch_upload_openai(file_paths)
file_ids = [
    'file-DprfWNMdHJtBzkKdraDZAx',
    'file-Tgf1CcLa5r7VcVNLbwaNmJ',
    'file-8zETMDPgMtmGWE4FCEAK7z'
]
file_id = file_ids[0]
# vector_store_id = vector_store_create()
vector_store_id = 'vs_6829a9ff40448191b62872f397c9a600'

In [74]:
vector_store_file_create(file_id, vector_store_id)

VectorStoreFile(id='file-DprfWNMdHJtBzkKdraDZAx', created_at=1747562576, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_6829a9ff40448191b62872f397c9a600', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [75]:
vector_store_file_creation_status(file_id, vector_store_id)

'completed'

In [76]:
vector_store_file_creation_check_completion(file_id, vector_store_id)

True

In [77]:
vector_store_file_create_pole(file_id, vector_store_id)

'file-DprfWNMdHJtBzkKdraDZAx'

In [84]:
local_file_path = "./sample_data/deep_research_blog.pdf"
file_id = vector_store_file_upload_pole(local_file_path, vector_store_id)
file_id

'file-N7ZYNST6RtfRbP1AHFryGo'

In [85]:
vector_store_file_retrieve(file_id, vector_store_id)

VectorStoreFile(id='file-N7ZYNST6RtfRbP1AHFryGo', created_at=1747562832, last_error=None, object='vector_store.file', status='completed', usage_bytes=66539, vector_store_id='vs_6829a9ff40448191b62872f397c9a600', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [86]:
vector_store_file_update(file_id, vector_store_id, {"location": "us"})

In [93]:
vector_store_file_list(vector_store_id)

{'data': [{'id': 'file-N7ZYNST6RtfRbP1AHFryGo',
   'created_at': 1747562832,
   'last_error': None,
   'object': 'vector_store.file',
   'status': 'completed',
   'usage_bytes': 66539,
   'vector_store_id': 'vs_6829a9ff40448191b62872f397c9a600',
   'attributes': {'location': 'us'},
   'chunking_strategy': {'static': {'chunk_overlap_tokens': 400,
     'max_chunk_size_tokens': 800},
    'type': 'static'}},
  {'id': 'file-6MWxb554D2ejxuKgy7kfDM',
   'created_at': 1747562806,
   'last_error': None,
   'object': 'vector_store.file',
   'status': 'completed',
   'usage_bytes': 66539,
   'vector_store_id': 'vs_6829a9ff40448191b62872f397c9a600',
   'attributes': {},
   'chunking_strategy': {'static': {'chunk_overlap_tokens': 400,
     'max_chunk_size_tokens': 800},
    'type': 'static'}},
  {'id': 'file-N1EmwEDDYthVLGUe8FGM3z',
   'created_at': 1747562705,
   'last_error': None,
   'object': 'vector_store.file',
   'status': 'completed',
   'usage_bytes': 66539,
   'vector_store_id': 'vs_6829a

In [95]:
vector_store_file_delete(file_id, vector_store_id)

VectorStoreFileDeleted(id='file-N7ZYNST6RtfRbP1AHFryGo', deleted=True, object='vector_store.file.deleted')

In [98]:
vector_store_file_delete(file_id, vector_store_id)

'Err: Error code: 404 - {\'error\': {\'message\': "No file found with id \'file-N7ZYNST6RtfRbP1AHFryGo\' in vector store \'vs_6829a9ff40448191b62872f397c9a600\'.", \'type\': \'invalid_request_error\', \'param\': None, \'code\': None}}'

## vector store batch operations

## generation

In [137]:
%time
print()

CPU times: user 13 μs, sys: 2 μs, total: 15 μs
Wall time: 35.5 μs



In [142]:
%%time
generate_with_file_search(
    query="What is deep research by OpenAI?", 
    vector_store_id=vector_store_id,
)

CPU times: user 30.1 ms, sys: 1.01 ms, total: 31.1 ms
Wall time: 9.71 s


Response(id='resp_682abf436ff08191a4b2e55170bb297d03d79a5751f7dba7', created_at=1747631939.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFileSearchToolCall(id='fs_682abf43d4648191afda528fcd6ce38903d79a5751f7dba7', queries=['deep research by OpenAI', 'What is deep research by OpenAI?'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_682abf46216481919adbee6bce692e9303d79a5751f7dba7', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-HYRTF6hP2UwEci3hHzUSzf', index=528, type='file_citation', filename='deep_research_blog.pdf'), AnnotationFileCitation(file_id='file-HYRTF6hP2UwEci3hHzUSzf', index=769, type='file_citation', filename='deep_research_blog.pdf'), AnnotationFileCitation(file_id='file-HYRTF6hP2UwEci3hHzUSzf', index=952, type='file_citation', filename='deep_research_blog.pdf'), AnnotationFileCitation(file_id='file-HYRTF6hP

In [136]:
generate_with_file_search(
    query="what do you know about task decomposition?", 
    vector_store_id=vector_store_id,
)


KeyboardInterrupt



In [146]:
%%time
generate_with_information_retrieval(
    query="What is deep research by OpenAI?", 
    vector_store_id=vector_store_id,
)

<sources><result file_id='file-HYRTF6hP2UwEci3hHzUSzf' file_name='deep_research_blog.pdf'><content>Introducing deep research | OpenAI


February 2, 2025 Release

Introducing deep research
An agent that uses reasoning to synthesize large amounts of
online information and complete multi-step research tasks
for you. Available to Pro users today, Plus and Team next.

Try on ChatGPT

Listen to article 8:18 Share

21/02/2025, 19:58 Introducing deep research | OpenAI

https://openai.com/index/introducing-deep-research/ 1/38

https://openai.com/research/index/release/
https://chatgpt.com/
https://openai.com/


Today we’re launching deep research in ChatGPT, a new agentic capability that
conducts multi-step research on the internet for complex tasks. It accomplishes in
tens of minutes what would take a human many hours.

Deep research is OpenAI's next agent that can do work for you independently—you
give it a prompt, and ChatGPT will find, analyze, and synthesize hundreds of online
sources to c

ChatCompletion(id='chatcmpl-BYnD6ka5TzpGaBM6QsWtZRFnGWy1N', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Deep research is a new capability introduced by OpenAI that allows an agent in ChatGPT to conduct multi-step research tasks by synthesizing large amounts of online information. It operates independently to find, analyze, and synthesize data from hundreds of sources, creating comprehensive reports similar to those produced by a research analyst. This feature is designed for intensive knowledge work in fields like finance, science, policy, and engineering, as well as for personalized shopping recommendations. It leverages a version of OpenAI's o3 model optimized for web browsing and data analysis, and is particularly effective at finding niche information. Users can request specific reports and receive a detailed output supported by citations and summaries of the research process. Deep research is available to Pro users and will 

In [134]:
generate_with_information_retrieval(
    query="what do you know about task decomposition?", 
    vector_store_id=vector_store_id,
)

ChatCompletion(id='chatcmpl-BYVlITUJnEKp7rC2GxYljKiHM4E2l', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I don't know.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1747564944, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_dbaca60df0', usage=CompletionUsage(completion_tokens=4, prompt_tokens=7772, total_tokens=7776, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [112]:
# rag_client = RAG_Client()
# rag_client.file_search("What is deep research by OpenAI?")

# real example

In [149]:
from glob import glob

In [147]:
vector_store_id = vector_store_create(name="salamat")
vector_store_id

'vs_682ac0ffd6e88191af5d28221bccf376'

In [148]:
vector_store_file_list(vector_store_id)

{'data': [],
 'has_more': False,
 'object': 'list',
 'first_id': None,
 'last_id': None}

In [151]:
file_paths = glob("./sample_data/salamat/*")
file_paths

['./sample_data/salamat/سرپایی.docx',
 './sample_data/salamat/نظارت.docx',
 './sample_data/salamat/بیمه گری.docx',
 './sample_data/salamat/دارویی.docx',
 './sample_data/salamat/صعب العلاج.docx',
 './sample_data/salamat/نظام ارجاع و پزشک خانواده.docx',
 './sample_data/salamat/خسارت متفرقه.docx',
 './sample_data/salamat/نسخه الکترونیک.docx',
 './sample_data/salamat/بستری.docx']

In [152]:
file_ids = batch_upload_openai(file_paths)
file_ids

['file-Jn8xQQ5Xd7aEcGx41SDPzS',
 'file-D5FcY9xenQ6FTaVRZJPQnK',
 'file-KfEPtkzqZFQhmXABDyg9g9',
 'file-Y8pRY2GsgTVmy7hrGq6ksz',
 'file-KZFeYHM1LpfkqBh3bV6GfA',
 'file-6JMoEeGGpEtc2RMShUtgbX',
 'file-Wrf49VJc9k4KKmj49yf9Gt',
 'file-VLGFReAFJFXgL5sL8da2pW',
 'file-PK9WpX7G84YCrmFbJaLH6h']

In [153]:
batch_id = batch_create_and_pole(vector_store_id, file_ids)

'vsfb_5c41e4c20adc495fa79475e3e6b9e300'

In [156]:
vector_store_file_list(vector_store_id)

{'data': [{'id': 'file-PK9WpX7G84YCrmFbJaLH6h',
   'created_at': 1747632528,
   'last_error': None,
   'object': 'vector_store.file',
   'status': 'completed',
   'usage_bytes': 8565,
   'vector_store_id': 'vs_682ac0ffd6e88191af5d28221bccf376',
   'attributes': {},
   'chunking_strategy': {'static': {'chunk_overlap_tokens': 400,
     'max_chunk_size_tokens': 800},
    'type': 'static'}},
  {'id': 'file-VLGFReAFJFXgL5sL8da2pW',
   'created_at': 1747632528,
   'last_error': None,
   'object': 'vector_store.file',
   'status': 'completed',
   'usage_bytes': 19450,
   'vector_store_id': 'vs_682ac0ffd6e88191af5d28221bccf376',
   'attributes': {},
   'chunking_strategy': {'static': {'chunk_overlap_tokens': 400,
     'max_chunk_size_tokens': 800},
    'type': 'static'}},
  {'id': 'file-Wrf49VJc9k4KKmj49yf9Gt',
   'created_at': 1747632528,
   'last_error': None,
   'object': 'vector_store.file',
   'status': 'completed',
   'usage_bytes': 13267,
   'vector_store_id': 'vs_682ac0ffd6e88191af5d28

In [158]:
%%time
generate_with_file_search(query="تعهدات ivf چجوریه؟", vector_store_id=vector_store_id)

CPU times: user 31.7 ms, sys: 5 ms, total: 36.8 ms
Wall time: 7.95 s


Response(id='resp_682ac2df51048191bbfb753e47cf693e058ccb2ab66ffe19', created_at=1747632863.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFileSearchToolCall(id='fs_682ac2e002a881919250dcf580fdda55058ccb2ab66ffe19', queries=['تعهدات ivf'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_682ac2e1dd1081919538a7865e031e3f058ccb2ab66ffe19', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-PK9WpX7G84YCrmFbJaLH6h', index=296, type='file_citation', filename='بستری.docx')], text='تعهدات IVF در صندوق\u200cهای بیمه سلامت به این شکل است:\n\n1. **در مراکز دولتی:** 90% تعرفه مصوب بخش دولتی.\n2. **در مراکز عمومی غیردولتی:** 90% تعرفه مصوب بخش عمومی غیردولتی.\n3. **در مراکز خصوصی و خیریه:** 70% تعرفه مصوب بخش خصوصی و خیریه.\n\nبیمه شدگان می\u200cتوانند از 4 بسته خدمتی در این مراکز استفاده کنند.', type='output_text')], role='assistant', statu

In [159]:
%%time
generate_with_information_retrieval(query="تعهدات ivf چجوریه؟", vector_store_id=vector_store_id)

<sources><result file_id='file-PK9WpX7G84YCrmFbJaLH6h' file_name='بستری.docx'><content>بستری

پرسش:

تعهدات IVF در صندوق های بیمه سلامت در مراکز طرف قرارداد و غیر طرف قرارداد به چه صورت می باشد ؟

پاسخ:

1-در مراکز دولتی با90% تعرفه مصوب بخش دولتی 2- در مراکز عمومی غیردولتی با 90%تعرفه مصوب بخش عمومی غیردولتی 3- درمراکز خصوصی وخیریه با 70% تعرفه مصوب بخش خصوصی وخیریه می توانند از 4 بسته خدمتی استفاده نمایند.

پرسش:

 فرانشیز خدمات بستری جهت مددجویان بهزیستی و کمیته امداد و همچنین روستاییان با رعایت نظام ارجاع صفر می باشد آیا این موضوع صرفا مربوط به مراکز دولتی و دانشگاهی بوده یا اینکه در مراکز دولتی غیر دانشگاهی نیز اعمال گردیده است؟

پاسخ:

  بر اساس مصوبه هیأت محترم وزیران ، فرانشیز مددجویان بهزیستی و کمیته امداد و همچنین روستاییان در صورت دریافت خدمات بستری و بستری فوریت های پزشکی(اورژانس)(دارای پرونده) در مراکز دولتی دانشگاهی در قالب  نظام ارجاع صفر می باشد.

پرسش:

هزینه اقامت (هتلینگ) اعلام شده در خصوص خدمات بستری به ازای هر شب بستری شدن بیمه شده می باشد یا برای کل دوران بستری مح

ChatCompletion(id='chatcmpl-BYnR3zpaD9B98iJ85S2ImGAjDmOER', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='تعهدات IVF در صندوق\u200cهای بیمه سلامت به شرح زیر است:\n\n1. در مراکز دولتی: 90% تعرفه مصوب بخش دولتی.\n2. در مراکز عمومی غیردولتی: 90% تعرفه مصوب بخش عمومی غیردولتی.\n3. در مراکز خصوصی و خیریه: 70% تعرفه مصوب بخش خصوصی و خیریه.\n\nبیمه\u200cشدگان می\u200cتوانند از 4 بسته خدمات IVF استفاده کنند.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1747632881, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_0392822090', usage=CompletionUsage(completion_tokens=104, prompt_tokens=3738, total_tokens=3842, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_token